In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime
from functools import partial

todo_items = []

layout_medium = widgets.Layout(width='280px')
layout_large = widgets.Layout(width='300px')
layout_button = widgets.Layout(width='300px', height='36px')

# 전체 진행률 바
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='', layout=widgets.Layout(width='40%', height='24px'))
overall_progress_bar.style.bar_color = 'black'
overall_progress_label = widgets.Label(value='(0%)', layout=widgets.Layout(margin='0 10px'))

# 리스트 박스
todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []

    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_large)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=widgets.Layout(height='20px'))
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_large)
        progress_col = widgets.VBox([progress_bar, progress_slider], layout=layout_large)
        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_large)
        priority_label = widgets.Label(value=item['priority'], layout=layout_large)

        complete_btn = widgets.Button(description='완료✅', layout=widgets.Layout(width='70px', height='32px'))
        delete_btn = widgets.Button(description='삭제❌', layout=widgets.Layout(width='70px', height='32px'))
        complete_btn.style.button_color = '#d3d3d3'
        delete_btn.style.button_color = '#d3d3d3'
        button_box = widgets.HBox([complete_btn, delete_btn])

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        progress_slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        row = widgets.GridBox([
            widgets.Label('할 일'), title,
            widgets.Label('진행률'), progress_col,
            widgets.Label('입력 마감일'), due_label,
            widgets.Label('중요도'), priority_label,
            widgets.Label('작업'), button_box
        ], layout=widgets.Layout(grid_template_columns='120px auto', margin='10px 0'))
        todo_list_box.children += (row,)

title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_large)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_large)
due_input = widgets.DatePicker(layout=layout_large)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_large)
add_button = widgets.Button(description='추가', button_style='success', layout=layout_button)

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
            update_overall_progress()
            refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=widgets.Layout(width='145px', height='36px'))
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=widgets.Layout(width='145px', height='36px'))
save_btn.on_click(save_data)
load_btn.on_click(load_data)

display(widgets.VBox([
    widgets.Label(value='DAVID mk17 - UI 개선 (가로폭 조정, 라벨 추가, 버튼 확장, 검정 진행률)'),
    widgets.GridBox([
        widgets.Label('할 일'), title_input,
        widgets.Label('진행률'), progress_input,
        widgets.Label('마감일'), due_input,
        widgets.Label('중요도'), priority_input,
    ], layout=widgets.Layout(grid_template_columns='80px auto', margin='10px 0')),
    widgets.HBox([add_button], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center', margin='10px 0')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center')),
    todo_list_box
]))

update_overall_progress()
refresh_list()
